# <center> webscraping

### première partie du projet bloc 1

- Webscraping du site [www.allocine.fr](https://www.allocine.fr/films/)

![filtres](images/filtresSMALL.png)

## Sources :
[beautiful-soup-4](https://beautiful-soup-4.readthedocs.io/en/latest/)<br>
Les liens sont sûrement générés aléatoirement dynamiquement, on peut utiliser XPath avec selenium<br>
ou bien avec lxml ??<br>
[beautiful-soup-4.readthedocs.io](https://beautiful-soup-4.readthedocs.io/en/latest/#searching-the-tree)<br>

[selenium-python.readthedocs.io](https://selenium-python.readthedocs.io/locating-elements.html)<br>
[selenium.dev/documentation](https://www.selenium.dev/documentation/webdriver/elements/information/)<br>
<br>

In [ ]:
%reset

In [153]:
import os
import re
#import httpx
import requests
import pandas as pd

from bs4 import BeautifulSoup
from IPython.display import display
from tqdm import tqdm

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options = options)

%config IPCompleter.greedy = True

url_site = 'https://www.allocine.fr/'
url_films = 'https://www.allocine.fr/films/'

### On scrape tous les genres de film

In [ ]:
# Scrap all categories
r = requests.get(url_films, auth=('user', 'pass'))
if r.status_code != 200:
    print("url_site error")
    
soup = BeautifulSoup(r.content, 'html.parser')
print(type(soup))

categories = []
eltCategories = soup.find('div', class_='filter-entity-section')
for elt in eltCategories.find_all('li'):
    categories.append(elt.a.text)

df_categories = pd.Series(categories)

dict_n_cat = {k:v for k, v in enumerate(categories)}
dict_cat_n = {v:k for v, k in dict_n_cat.items()}

<class 'bs4.BeautifulSoup'>


### On scrapte les films par période
[1980 - 1989] puis [1990 - 1999] ...

In [3]:
# Scrap url of years we want to scrap the movies
# Not Working
# I cannot get the url by scraping
decades_to_scrap = ['1980 - 1989']
eltYears = eltCategories.find_next_sibling()
lstUrl = []
eltCur = eltYears.find('li')

while eltCur:
    text = eltCur.span.text
    # print(eltCur)
    # print(eltCur.span.get('href'))
    if text in decades_to_scrap:
        print("**", text)
    else:
        print(eltCur.span.text)
    eltCur = eltCur.find_next_sibling()

2030 - 2039
2020 - 2029
2010 - 2019
2000 - 2009
1990 - 1999
** 1980 - 1989
1970 - 1979
1960 - 1969
1950 - 1959
1940 - 1949
1930 - 1939
1920 - 1929
1910 - 1919
1900 - 1909
1890 - 1899


### Ou bien nous pouvons entrer les url manuellement

In [4]:
url_decades = url_films + 'decennie-1980/'
url_year = url_films + 'decennie-1980/annee-1980/'

def getNumberOfPages(elt):
    nb = 0
    while elt:
        if elt.text.isdigit():
            nb = elt.text
        elt = elt.find_next_sibling()
    return int(nb)

r = requests.get(url_year, auth=('user', 'pass'))
if r.status_code != 200:
    print("url_site error")

# We get the number of pages for this year
soup = BeautifulSoup(r.content, 'html.parser')
pagination = soup.find('div', class_='pagination-item-holder')
nb_pages = int(pagination.find_all('span')[-1].text)
assert nb_pages == getNumberOfPages(pagination.find('span'))
print(nb_pages)

47


### On scrape chaque page de films pour une année donnée

In [ ]:
def scrapMoviePage(url_movie):
    # get soup from movie page
    r = requests.get(url_movie, auth=('user', 'pass'))
    soupMovie = BeautifulSoup(r.content, 'html.parser')
    print("Title:" , soupMovie.find('div', class_ = "titlebar-title titlebar-title-xl").text)
    
    elt = soupMovie.find('div', class_="meta-body-item meta-body-info")
    text = elt.get_text(strip=True)

    s1, s2, s3 = text.split('|')
    date = s1[:-8].strip()
    print("Date:", date)
    duration = s2.strip()
    print("Duration:", duration)
    categories = s3.strip()
    print("Categories:", categories)

    elts = soupMovie.find_all('div', class_ = "meta-body-item meta-body-direction meta-body-oneline")
    #assert len(elts)
    authors = [elts[0].text.strip()[2:].strip()]

    if len(elts) > 1:
        elts_span = elts[1].find_all('span')
        for elt in elts_span:
            if 'light' in elt['class']:
                continue
            if elt.get_text(strip=True) not in authors:
                authors.append(elt.get_text(strip=True))
    authors = ', '.join(authors)
    print("Authors:", authors)

    elt = soupMovie.find('div', class_ = "meta-body-item meta-body-actor")
    actors = elt.get_text(strip=True)[4:]
    print("Actors:", actors)

    elt = soupMovie.find('section', class_ = "section ovw ovw-synopsis")
    elt2 = elt.find('div', class_ = "content-txt")
    elt3 = elt.find('p', class_ = 'bo-p')
    summary = elt3.text.strip()
    print("Summary:", summary)

    # driver to the movie page
    driver.get(url_movie)
    



    return
    # beautiful soup version
    elts_rating = soupMovie.find_all('div', class_ = 'rating-item')
    #print(len(elts_rating))
    ratings = {}

    for elt_rating in elts_rating:
        print(elt_rating.prettify())
        elt_temp = elt_rating.find('div', class_='rating-item-content')
        elt_span = elt_temp.find('span')
        print("span class", elt_span['class'])
        if 'rating-title' in elt_span['class']:
            print('rating-title')
            print(elt_span.get_text(strip = True))
            if 'Spectateurs' in elt_span.get_text(strip = True):
                ratings['spectateurs'] = elt_temp.find('span', class_ = 'stareval-note').text
        #     # print(elt_span.get_text(strip = True))
        # elif 'rating-title' in elt_span['class']:
        #     if 'Presse' in elt_span.get_text(strip = True):
        #         ratings['Presse'] = elt_temp.find('span', class_ = 'stareval-note').text
    print("ratings:", ratings)

movies = []

for i in range(1, 3):
    url_year_page = url_year + '?page=' + str(i)
    r = requests.get(url_year_page, auth=('user', 'pass'))
    if r.status_code != 200:
        print("url_site error")
    soupMovies = BeautifulSoup(r.content, 'html.parser')
    eltMovies = soupMovies.find_all('li', class_='mdl')

    for eltMovie in eltMovies:
        url_movie = url_site + eltMovie.h2.a.get('href')
        scrapMoviePage(url_movie)
    #    print()
    #break

Title: La Boum
Title: Shining
Title: Elephant Man
Title: Star Wars : Episode V - L'Empire contre-attaque
Title: American Gigolo
Title: Cannibal Holocaust
Title: Les Sous-doués
Title: Le Roi et l'oiseau
Title: Y a-t-il un pilote dans l'avion ?
Title: Raging Bull
Title: Le Dernier métro
Title: Pulsions
Title: The Blues Brothers
Title: Beau-Père
Title: Je vais craquer
Title: La Chasse - Cruising
Title: Le Miroir se brisa
Title: Le Lagon Bleu
Title: La Porte du paradis
Title: Popeye
Title: Ça va cogner
Title: Le Petit Lord Fauntleroy
Title: Vendredi 13
Title: Pile ou face
Title: Gloria
Title: Au-delà du réel
Title: La Banquière
Title: La Femme flic
Title: Kagemusha, l'ombre du guerrier
Title: Fog


### On scrape les pays

In [ ]:
eltCountries = eltYears.find_next_sibling()
print(eltCountries)

In [ ]:

driver.get("https://www.allocine.fr/film/fichefilm_gen_cfilm=863.html")

# elem = driver.find_element(By.NAME, "q")
# elem.clear()
# elem.send_keys("pycon")
# elem.send_keys(Keys.RETURN)


# elts = driver.find_elements(By.CLASS_NAME, 'xXx rating-title')
elts = driver.find_elements(By.CLASS_NAME, 'rating-item')
print(len(elts))
for elt in elts[:2]:
    print(elt.text)
    # print(elt.get_attribute('href'))
    elt_a = elt.find_element(By.TAG_NAME, 'a')
    print(elt_a.get_attribute('href'))
#elts_rating = elt.find_elements(By.XPATH, '/html/body/div[2]/main/section/div/div[3]/div[2]/div')

# for elt in elts_rating:
#     elt_temp = elt.find_element(By.CLASS_NAME, 'rating-item-content')
#     elt_a = driver.find_element(By.TAG_NAME, 'a')
    # elt_a = elt_temp.find_element(By.CLASS_NAME, 'xXx')
    # elt_a = elt_temp.find_element(By.CLASS_NAME, 'rating-title')
    # elt_a = elt.find_element(By.CLASS_NAME, 'xXx rating-title')
    # print(elt_a.get_attribute('href'))

3
Presse
4,0
5 critiques
https://www.allocine.fr/film/fichefilm-863/critiques/presse/
Spectateurs
4,3
58968 notes, 1750 critiques
https://www.allocine.fr/film/fichefilm-863/critiques/spectateurs/


In [ ]:
driver.close()